### Тестирование с помощью LLM As Judge

In [1]:
from json import load

In [2]:
#!pip install langchain langchain-core langchain-mistralai pydantic

In [3]:
with open('val_retrieve_responses.json', 'r') as f:
    val_retrieve_responses = load(f)
with open('val_generate_answers.json', 'r') as f:
    val_generate_answers = load(f)
with open('val_generate_ground_truths.json', 'r') as f:
    val_generate_ground_truths = load(f)
with open('val_queries.json', 'r') as f:
    val_queries = load(f)

In [4]:
limit = 3
retriever_contexts = {}
for k, v in val_retrieve_responses.items():
    contexts = []
    for i in range(limit):
        context = v[i]['content']
        contexts.append(context)
    retriever_contexts[k] = contexts

In [5]:
import dotenv
import os
dotenv.load_dotenv('.env.host')

os.environ['MISTRAL_API_KEY'] = os.getenv('LLM_API_KEY')

In [6]:

from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

judge = ChatMistralAI(
    model="mistral-large-latest",  # или другой judge
    temperature=0
)

In [7]:
import numpy as np
from typing import List
from pydantic import BaseModel, Field


from tqdm import tqdm

# ---------- Faithfulness ----------
class FaithfulnessResult(BaseModel):
    faithfulness: float = Field(ge=0.0, le=1.0)
    unsupported_claims: List[str] = Field(default_factory=list)

faith_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a strict RAG evaluator. Use ONLY the provided context. "
     "If the answer contains any factual claim not supported by the context, it is unsupported. "
     "Return JSON only."),
    ("human",
     "Question:\n{question}\n\n"
     "Context:\n{context}\n\n"
     "Answer:\n{answer}\n\n"
     "Output JSON with faithfulness in [0,1] and unsupported_claims.")
])

faith_chain = faith_prompt | judge.with_structured_output(FaithfulnessResult)

# ---------- Correctness ----------
class CorrectnessResult(BaseModel):
    correctness: float = Field(ge=0.0, le=1.0)
    major_errors: List[str] = Field(default_factory=list)

corr_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a strict evaluator. Compare the answer to the ground truth. "
     "Return JSON only."),
    ("human",
     "Question:\n{question}\n\n"
     "Ground truth:\n{ground_truth}\n\n"
     "Answer:\n{answer}\n\n"
     "Output JSON with correctness in [0,1] and major_errors.")
])

corr_chain = corr_prompt | judge.with_structured_output(CorrectnessResult)

# ---------- Dataset-level scoring ----------
def eval_dataset(qids, questions, contexts, answers, ground_truths, tau=0.8):
    faith_scores, corr_scores = [], []

    for qid in tqdm(qids):
        ctx = "\n\n---\n\n".join(contexts[qid])  # top-3 chunks as strings
        ans = answers[qid]
        q   = questions[qid]
        gt  = ground_truths[qid]

        faith = faith_chain.invoke({"question": q, "context": ctx, "answer": ans})
        corr  = corr_chain.invoke({"question": q, "ground_truth": gt, "answer": ans})

        faith_scores.append(faith.faithfulness)
        corr_scores.append(corr.correctness)

    mean_faith = float(np.mean(faith_scores))
    mean_corr  = float(np.mean(corr_scores))
    task_success = float(np.mean([c >= tau for c in corr_scores]))
    halluc_rate  = 1.0 - mean_faith

    return {
        "mean_correctness": mean_corr,
        "task_success_rate": task_success,
        "mean_faithfulness": mean_faith,
        "hallucination_rate": halluc_rate,
    }


In [8]:
val_queries

{'3a004357-19ab-4ed6-b60c-02e419657cc4': {'query': 'What is the concept of location-class independence in object detection?',
  'type': 'abstractive',
  'source': 'text-image'},
 '0dd5fa73-c74a-43cd-815b-5c1cb8d3d50a': {'query': 'Why is post-calibrated softmax adjustment (PCSA) important in handling class imbalances?',
  'type': 'abstractive',
  'source': 'text-image'},
 'be608e3f-d284-4b13-b37d-5dd2571a8572': {'query': 'What does the term $p(y, o, u)$ represent in object detection?',
  'type': 'extractive',
  'source': 'text'},
 'ed761078-1f4b-45ec-a3f1-7084d2825171': {'query': 'Is there any significant relationship between geolocation error and performance in the updated GEDI dataset?',
  'type': 'extractive',
  'source': 'text'},
 '5a7d0a0b-d63d-4095-8c50-6a2f8f453554': {'query': 'How do bootstrap variance estimators compare to true variances in statistical studies?',
  'type': 'abstractive',
  'source': 'text-table-image'},
 'dffc3d17-6be2-4ce4-81b3-5ca9d57d8abb': {'query': 'How is

In [9]:
import dotenv
import os
dotenv.load_dotenv('.env.host')

os.environ['MISTRAL_API_KEY'] = os.getenv('LLM_API_KEY')

In [10]:
print(judge.invoke("Say OK"))

content='OK! 😊' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 5, 'total_tokens': 11, 'completion_tokens': 6}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop', 'model_provider': 'mistralai'} id='lc_run--019b1fc9-37c7-7441-8ae7-3a4b13d7a779-0' usage_metadata={'input_tokens': 5, 'output_tokens': 6, 'total_tokens': 11}


In [14]:
questions

{'3a004357-19ab-4ed6-b60c-02e419657cc4': 'What is the concept of location-class independence in object detection?',
 '0dd5fa73-c74a-43cd-815b-5c1cb8d3d50a': 'Why is post-calibrated softmax adjustment (PCSA) important in handling class imbalances?',
 'be608e3f-d284-4b13-b37d-5dd2571a8572': 'What does the term $p(y, o, u)$ represent in object detection?',
 'ed761078-1f4b-45ec-a3f1-7084d2825171': 'Is there any significant relationship between geolocation error and performance in the updated GEDI dataset?',
 '5a7d0a0b-d63d-4095-8c50-6a2f8f453554': 'How do bootstrap variance estimators compare to true variances in statistical studies?',
 'dffc3d17-6be2-4ce4-81b3-5ca9d57d8abb': 'How is land cover classified in the Copernicus Global Land Service map?',
 'a078059a-08f7-4bde-8488-329b3db39123': 'How does the structure multivector (SMV) improve spatial phase estimation?',
 '637fdb28-12fb-472d-8d35-ea68a4ab77f6': 'Why is accurate estimation of Above Ground Biomass important for global challenges?

In [11]:
questions = {k: v['query'] for k, v in val_queries.items()}

#для скорости возьмем первые 50 вопросов
result = eval_dataset(qids=list(questions.keys())[:50],
             questions=questions,
             contexts=retriever_contexts,
             answers=val_generate_answers,
             ground_truths=val_generate_ground_truths,
             tau=0.8)

100%|██████████| 50/50 [05:06<00:00,  6.13s/it]


In [16]:
result

{'mean_correctness': 0.5299999999999999,
 'task_success_rate': 0.22,
 'mean_faithfulness': 0.424,
 'hallucination_rate': 0.5760000000000001}

Здесь пока не очень удовлетворяющие нас значения метрик.